### 2016 CSD

In [2]:
import duckdb

# Path to your CSV file
csv_file_path = '/home/mac/code/municipalities/98-401-X2016055_English_CSV_data.csv'

# Connect to DuckDB (it will create a database file in the current directory)
conn = duckdb.connect('can.db')

# Load CSV data into a DuckDB table
conn.execute(f"""
    DROP TABLE csd2016;
    CREATE TABLE csd2016 AS 
    SELECT * 
    FROM read_csv_auto('{csv_file_path}')
""")

# Verify that the table has been created and contains data
result = conn.execute("SELECT * FROM csd2016 LIMIT 5").fetchall()
print(result)

# Close the connection
conn.close()

[(2016, '01', 0, 'Canada', 4.0, 5.1, '20000', ' ', '01', 'Population, 2016', 1, 1, '35151728', '...', '...'), (2016, '01', 0, 'Canada', 4.0, 5.1, '20000', ' ', '01', 'Population, 2011', 2, 2, '33476688', '...', '...'), (2016, '01', 0, 'Canada', 4.0, 5.1, '20000', ' ', '01', 'Population percentage change, 2011 to 2016', 3, None, '5.0', '...', '...'), (2016, '01', 0, 'Canada', 4.0, 5.1, '20000', ' ', '01', 'Total private dwellings', 4, 3, '15412443', '...', '...'), (2016, '01', 0, 'Canada', 4.0, 5.1, '20000', ' ', '01', 'Private dwellings occupied by usual residents', 5, 4, '14072079', '...', '...')]


In [3]:
# Connect to the DuckDB database
conn = duckdb.connect('can.db')

# Use DESCRIBE to get the schema of the table
describe_result = conn.execute("DESCRIBE csd2016").fetchall()
print("DESCRIBE result:")
for row in describe_result:
    print(row)


# Close the connection
conn.close()

DESCRIBE result:
('CENSUS_YEAR', 'BIGINT', 'YES', None, None, None)
('GEO_CODE (POR)', 'VARCHAR', 'YES', None, None, None)
('GEO_LEVEL', 'BIGINT', 'YES', None, None, None)
('GEO_NAME', 'VARCHAR', 'YES', None, None, None)
('GNR', 'DOUBLE', 'YES', None, None, None)
('GNR_LF', 'DOUBLE', 'YES', None, None, None)
('DATA_QUALITY_FLAG', 'VARCHAR', 'YES', None, None, None)
('CSD_TYPE_NAME', 'VARCHAR', 'YES', None, None, None)
('ALT_GEO_CODE', 'VARCHAR', 'YES', None, None, None)
('DIM: Profile of Census Divisions/Census Subdivisions (2247)', 'VARCHAR', 'YES', None, None, None)
('Member ID: Profile of Census Divisions/Census Subdivisions (2247)', 'BIGINT', 'YES', None, None, None)
('Notes: Profile of Census Divisions/Census Subdivisions (2247)', 'BIGINT', 'YES', None, None, None)
('Dim: Sex (3): Member ID: [1]: Total - Sex', 'VARCHAR', 'YES', None, None, None)
('Dim: Sex (3): Member ID: [2]: Male', 'VARCHAR', 'YES', None, None, None)
('Dim: Sex (3): Member ID: [3]: Female', 'VARCHAR', 'YES', Non

In [4]:
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect('can.db')

# Rename the columns
conn.execute("""
    ALTER TABLE csd2016 RENAME COLUMN "Dim: Sex (3): Member ID: [1]: Total - Sex" TO total;
    ALTER TABLE csd2016 RENAME COLUMN "Dim: Sex (3): Member ID: [2]: Male" TO male;
    ALTER TABLE csd2016 RENAME COLUMN "Dim: Sex (3): Member ID: [3]: Female" TO female;
    ALTER TABLE csd2016 RENAME COLUMN "DIM: Profile of Census Divisions/Census Subdivisions (2247)" TO CHARACTERISTIC_NAME;
    ALTER TABLE csd2016 RENAME COLUMN "Member ID: Profile of Census Divisions/Census Subdivisions (2247)" TO CHARACTERISTIC_ID;
    ALTER TABLE csd2016 RENAME COLUMN "Notes: Profile of Census Divisions/Census Subdivisions (2247)" TO Notes;
""")

# Update the values equal to character to 0
conn.execute("""
    UPDATE csd2016
    SET total = CASE WHEN total = 'x' THEN '0' ELSE total END,
        male = CASE WHEN male = 'x' THEN '0' ELSE male END,
        female = CASE WHEN female = 'x' THEN '0' ELSE female END
""")

conn.execute("""
    UPDATE csd2016
    SET total = CASE WHEN total = '...' THEN '0' ELSE total END,
        male = CASE WHEN male = '...' THEN '0' ELSE male END,
        female = CASE WHEN female = '...' THEN '0' ELSE female END;
""")

conn.execute("""
    UPDATE csd2016
    SET total = CASE WHEN total = 'F' THEN '0' ELSE total END,
        male = CASE WHEN male = 'F' THEN '0' ELSE male END,
        female = CASE WHEN female = 'F' THEN '0' ELSE female END;
""")

conn.execute("""
    UPDATE csd2016
    SET total = CASE WHEN total = '..' THEN '0' ELSE total END,
        male = CASE WHEN male = '..' THEN '0' ELSE male END,
        female = CASE WHEN female = '..' THEN '0' ELSE female END;
""")

# Create a temporary table to cast the renamed columns to FLOAT
conn.execute("""
    ALTER TABLE csd2016 ALTER total TYPE FLOAT;
    ALTER TABLE csd2016 ALTER male TYPE FLOAT;
    ALTER TABLE csd2016 ALTER female TYPE FLOAT;
""")

# Verify the changes

# Use DESCRIBE to get the schema of the table
describe_result = conn.execute("DESCRIBE csd2016").fetchall()
print("DESCRIBE result:")
for row in describe_result:
    print(row)
    
# Close the connection
conn.close()

DESCRIBE result:
('CENSUS_YEAR', 'BIGINT', 'YES', None, None, None)
('GEO_CODE (POR)', 'VARCHAR', 'YES', None, None, None)
('GEO_LEVEL', 'BIGINT', 'YES', None, None, None)
('GEO_NAME', 'VARCHAR', 'YES', None, None, None)
('GNR', 'DOUBLE', 'YES', None, None, None)
('GNR_LF', 'DOUBLE', 'YES', None, None, None)
('DATA_QUALITY_FLAG', 'VARCHAR', 'YES', None, None, None)
('CSD_TYPE_NAME', 'VARCHAR', 'YES', None, None, None)
('ALT_GEO_CODE', 'VARCHAR', 'YES', None, None, None)
('CHARACTERISTIC_NAME', 'VARCHAR', 'YES', None, None, None)
('CHARACTERISTIC_ID', 'BIGINT', 'YES', None, None, None)
('Notes', 'BIGINT', 'YES', None, None, None)
('total', 'FLOAT', 'YES', None, None, None)
('male', 'FLOAT', 'YES', None, None, None)
('female', 'FLOAT', 'YES', None, None, None)


import duckdb
conn = duckdb.connect('can.db')

#### Load CSV data into a DuckDB table
conn.execute(f"""
    DROP TABLE csd2016;
""")

#### Close the connection
conn.close()